<a href="https://colab.research.google.com/github/TechWhizGenius/Teja_INFO5731_Fall2024/blob/main/Mandaloju_Teja_Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [2]:

# Installing necessary packages
!apt-get update
!apt-get install -y wget unzip
!pip install selenium


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,027 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,447 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Package

In [3]:
# Install Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

# Check Chrome version
!google-chrome --version

# Downloading the correct version of ChromeDriver (matching Chrome 128.0.6613.137)
!wget https://storage.googleapis.com/chrome-for-testing-public/128.0.6613.137/linux64/chromedriver-linux64.zip

# Unzip ChromeDriver directly to /usr/local/bin
!unzip chromedriver-linux64.zip -d /usr/local/bin/

# Move the extracted chromedriver to the correct location and set permissions
!sudo mv /usr/local/bin/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
!sudo chmod +x /usr/local/bin/chromedriver

# Verify ChromeDriver installation
!chromedriver --version



--2024-10-03 00:34:49--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 172.217.214.93, 172.217.214.136, 172.217.214.91, ...
Connecting to dl.google.com (dl.google.com)|172.217.214.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111878140 (107M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 106.69M  55.0MB/s    in 1.9s    

2024-10-03 00:34:51 (55.0 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [111878140/111878140]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 123620 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (129.0.6668.89-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Packag

In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Set up Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set up the ChromeDriver service (use your correct path)
service = Service('/usr/local/bin/chromedriver')

# Initialize WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Define the function to fetch IMDb reviews
def fetch_imdb_reviews(movie_url, num_reviews):
    driver.get(movie_url)

    reviews = []

    while len(reviews) < num_reviews:
        try:
            # Wait for review elements to be present
            review_elements = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.lister-item'))
            )

            for review_element in review_elements:
                if len(reviews) >= num_reviews:
                    break
                try:
                    # Extract review title
                    review_title = review_element.find_element(By.CSS_SELECTOR, '.title').text.strip()
                    # Extract review text
                    review_text = review_element.find_element(By.CSS_SELECTOR, '.text.show-more__control').text.strip()
                    # Extract rating (if available)
                    try:
                        rating_element = review_element.find_element(By.CSS_SELECTOR, '.rating-other-user-rating span')
                        rating = rating_element.text.strip().split("/")[0]  # Get the numeric part before '/'
                    except NoSuchElementException:
                        rating = "No Rating"

                    reviews.append({
                        'Review Title': review_title,
                        'Review Text': review_text,
                        'Rating': rating
                    })

                except Exception as e:
                    print(f"Error extracting review info: {e}")

            try:
                # Click the 'Load More' button to load more reviews
                load_more_button = driver.find_element(By.CSS_SELECTOR, '.load-more-data')
                load_more_button.click()
                time.sleep(2)  # Allow time for the new reviews to load
            except NoSuchElementException:
                print("No more reviews or 'Load More' button not found.")
                break

        except TimeoutException:
            print("Timed out waiting for reviews.")
            break

    return reviews

# Fetch and save the data
movie_url = 'https://www.imdb.com/title/tt15398776/reviews'  # Example movie URL (Oppenheimer)
num_reviews = 1000  # Number of reviews to scrape
reviews = fetch_imdb_reviews(movie_url, num_reviews)

# Convert to DataFrame and save to CSV
df = pd.DataFrame(reviews)
df.to_csv('imdb_reviews.csv', index=False)

# Quit the WebDriver
driver.quit()

print(f"Scraped {len(reviews)} reviews and saved to 'imdb_reviews.csv'")
print(df.head(10))


Scraped 1000 reviews and saved to 'imdb_reviews.csv'
                                        Review Title  \
0                              Murphy is exceptional   
1  A challenging watch to be sure, but a worthwhi...   
2                             Quality but exhausting   
3                           And the Oscar goes to...   
4  A brilliantly layered examination of a man thr...   
5                                    Is it just me ?   
6      Nolan touches greatness, falls slightly short   
7                                 Severely overhyped   
8       A Cinematic Masterpiece by Christopher Nolan   
9                                      A Masterpiece   

                                         Review Text     Rating  
0  You'll have to have your wits about you and yo...          9  
1                                                             8  
2  I'm a big fan of Nolan's work so was really lo...          7  
3  I'm still collecting my thoughts after experie...         10  


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [21]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

#df = pd.read_csv('imdb_reviews.csv')

# Initialize the stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if not isinstance(text, str):
        return ""  # Return empty string for non-string inputs
    text = re.sub(r'[^\w\s]', '', text)     # Remove noise
    text = re.sub(r'\d+', '', text)    # Remove numbers
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word.lower() not in stop_words)     # Remove stopwords
    text = text.lower()                                                       # lowercasing all texts
    text = ' '.join(stemmer.stem(word) for word in text.split())     # Stemming
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())    # Lemmatization

    return text

# Apply the cleaning function to the 'Review Text' column
df['Cleaned Review Text'] = df['Review Text'].apply(clean_text)
df['Cleaned Review Title'] = df['Review Title'].apply(clean_text)

# Save the cleaned data to a new CSV file
df.to_csv('imdb_reviews_cleaned.csv', index=False)

df.to_csv('imdb_reviews_cleaned.csv', index=False)

print(df[['Review Text', 'Cleaned Review Text']].head(10))
print(df[['Review Title', 'Cleaned Review Title']].head(10))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                         Review Text  \
0  You'll have to have your wits about you and yo...   
1                                                      
2  I'm a big fan of Nolan's work so was really lo...   
3  I'm still collecting my thoughts after experie...   
4  "Oppenheimer" is a biographical thriller film ...   
5  Is it just me or did anyone else find this mov...   
6                                                      
7  I align with other reviewers here who have fou...   
8  After a busy career filled with masterpieces, ...   
9  I may consider myself lucky to be alive to wat...   

                                 Cleaned Review Text  
0  youll wit brain fulli switch watch oppenheim c...  
1                                                     
2  im big fan nolan work realli look forward unde...  
3  im still collect thought experienc film cillia...  
4  oppenheim biograph thriller film written direc...  
5  anyon el find movi hate say bore know know pie... 

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [8]:
# Your code here
!pip install nltk spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [23]:
import pandas as pd
import nltk
import spacy
from nltk import pos_tag, word_tokenize
from nltk.tree import Tree

#df = pd.read_csv('imdb_reviews_cleaned.csv')

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

nlp = spacy.load('en_core_web_sm') # Load the spaCy model

# Part (1): POS Tagging and Count
def pos_tagging_and_count(text):
    if isinstance(text, str):  # Check if the input is a string
        tokens = word_tokenize(text)
        tagged = pos_tag(tokens)
        counts = {
            'Noun': sum(1 for word, tag in tagged if tag.startswith('NN')),
            'Verb': sum(1 for word, tag in tagged if tag.startswith('VB')),
            'Adjective': sum(1 for word, tag in tagged if tag.startswith('JJ')),
            'Adverb': sum(1 for word, tag in tagged if tag.startswith('RB')),
        }
        return counts
    else:
        return {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}  # Return zero counts for non-strings

# Calculate POS counts for each review
df['POS Count Text'] = df['Cleaned Review Text'].apply(pos_tagging_and_count)
df['POS Count Title'] = df['Cleaned Review Title'].apply(pos_tagging_and_count)

# Combine counts into a single DataFrame
pos_summary01 = pd.DataFrame(df['POS Count Text'].tolist()).sum().reset_index()
pos_summary02 =pd.DataFrame(df['POS Count Title'].tolist()).sum().reset_index()
pos_summary01.columns = ['Part of Speech', 'Count']
pos_summary02.columns = ['Part of Speech', 'Count']
print("Parts of Speech Counts:")
print(pos_summary01)
print(pos_summary02)

# Part (2): Parsing Trees
def parsing_trees(text):
    if isinstance(text, str):  # Check if the input is a string
        doc = nlp(text)
        # Print the constituency parsing tree
        print("\nConstituency Parsing Tree:")
        for sent in doc.sents:
            print(sent.root)  # Printing the root node for simplicity

        # Dependency Parsing
        dependency_tree = [(token.text, token.dep_, token.head.text) for token in doc]
        return dependency_tree
    else:
        return []  # Return empty for non-strings

# Example for parsing
sample_text01 = df['Cleaned Review Text'].iloc[0]
dependency_tree01 = parsing_trees(sample_text01)

sample_text02 = df['Cleaned Review Title'].iloc[0]
dependency_tree02 = parsing_trees(sample_text02)

print("\nDependency Parsing Tree:")
print(dependency_tree01)  # Printing dependency relations
print(dependency_tree02)  # Printing dependency relations

# Part (3): Named Entity Recognition (NER)
def extract_entities(text):
    if isinstance(text, str):  # Check if the input is a string
        doc = nlp(text)
        entities = {}
        for ent in doc.ents:
            entities[ent.label_] = entities.get(ent.label_, 0) + 1
        return entities
    else:
        return {}  # Return empty dictionary for non-strings

# Extract entities for each review
df['Entities01'] = df['Cleaned Review Text'].apply(extract_entities)
df['Entities02'] = df['Cleaned Review Title'].apply(extract_entities)

# Summarizing the counts of entities
entity_summary01 = {}
for entity_counts in df['Entities01']:
    for entity, count in entity_counts.items():
        entity_summary01[entity] = entity_summary01.get(entity, 0) + count

print("\nNamed Entity Counts:")
print(entity_summary01)

entity_summary02 = {}
for entity_counts in df['Entities02']:
    for entity, count in entity_counts.items():
        entity_summary02[entity] = entity_summary02.get(entity, 0) + count

print("\nNamed Entity Counts:")
print(entity_summary02)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Parts of Speech Counts:
  Part of Speech  Count
0           Noun  55112
1           Verb  11739
2      Adjective  20748
3         Adverb   4557
  Part of Speech  Count
0           Noun   2254
1           Verb    229
2      Adjective    693
3         Adverb    126

Constituency Parsing Tree:
absolut
say

Constituency Parsing Tree:
murphi

Dependency Parsing Tree:
[('you', 'nsubj', 'wit'), ('ll', 'aux', 'wit'), ('wit', 'nsubj', 'absolut'), ('brain', 'compound', 'switch'), ('fulli', 'compound', 'switch'), ('switch', 'compound', 'watch'), ('watch', 'dobj', 'wit'), ('oppenheim', 'nsubj', 'easili'), ('could', 'aux', 'easili'), ('easili', 'ccomp', 'wit'), ('get', 'conj', 'wit'), ('away', 'prt', 'get'), ('nonattent', 'compound', 'filmmak'), ('viewer', 'compound', 'intellig'), ('intellig', 'compound', 'filmmak'), ('filmmak', 'nsubj', 'show'), ('show', 'conj', 'wit'), ('audienc', 'nmod', 'fire'), ('great', 'amod', 'respect'), ('respect', 'compound', 'fire'), ('fire', 'nmod', 'inform'), ('dialogu

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

In [24]:
df.to_csv('imdb_reviews_cleaned.csv', index=False)

print("Cleaned data saved to 'imdb_reviews_cleaned.csv'")

from google.colab import files
files.download('imdb_reviews_cleaned.csv')

Cleaned data saved to 'imdb_reviews_cleaned.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
#link to file: https://drive.google.com/file/d/1OGv-1dnitseHaLL4ZRriXApxNyvRPjNN/view?usp=sharing

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [9]:
# Write your response below

'''
This assignment is really good. At first, I have used selenium to scrape data from IMDB. Later, performed some data cleaning on the dataset.
Finally, Implemented POS tagging. I have enjoyed doing all the tasks. I have spent 3 hours to complete this assignment. It is really worth it
as it has deepened my knowledge on data cleaning techniques of a corpus.
'''

#

'\nThis assignment is really good. At first, I have used selenium to scrape data from IMDB. Later, performed some data cleaning on the dataset. \nFinally, Implemented POS tagging. I have enjoyed doing all the tasks. I have spent 3 hours to complete this assignment. It is really worth it \nas it has deepened my knowledge on data cleaning techniques of a corpus.\n'